In [17]:
from credentials import *
import requests
import json, csv


#REQUEST URL
#example
url = "https://api.data.gov/nrel/alt-fuel-stations/v1/nearest.json?api_key={}&location=Denver+CO".format(api_key)

response = requests.get(url)

data = response.json()
#print(data)


In [5]:
for k in data:
    print(k)

station_locator_url
latitude
offset
precision
fuel_stations
longitude
station_counts
total_results


In [8]:
def return_total_results(data):
    total = data['total_results']
    print("[*] you have {} results for your query...".format(total))
    return total



In [9]:
return_total_results(data)

[*] you have 71 results for your query...


71

In [10]:
stations = data['fuel_stations']

In [13]:
len(stations)

20

In [ ]:
def get_pagination(request):
    "pagination":{"page":12,"per_page":100,"count":1054,"pages":11}}
    

In [15]:
response.json()

{'fuel_stations': [{'access_days_time': '24 hours daily',
   'bd_blends': None,
   'cards_accepted': None,
   'city': 'Denver',
   'date_last_confirmed': '2017-09-23',
   'distance': 0.24074,
   'e85_blender_pump': None,
   'ev_connector_types': ['J1772'],
   'ev_dc_fast_num': None,
   'ev_level1_evse_num': None,
   'ev_level2_evse_num': 1,
   'ev_network': 'ChargePoint Network',
   'ev_network_ids': {'posts': ['1:183793']},
   'ev_network_web': 'http://www.chargepoint.com/',
   'ev_other_evse': None,
   'expected_date': None,
   'fuel_type_code': 'ELEC',
   'geocode_status': 'GPS',
   'groups_with_access_code': 'Public',
   'hy_status_link': None,
   'id': 85932,
   'intersection_directions': 'CULTURAL CTR 3A; On Level 3 near the elevators',
   'latitude': 39.7360835,
   'longitude': -104.9881709,
   'lpg_primary': None,
   'ng_fill_type_code': None,
   'ng_psi': None,
   'ng_vehicle_class': None,
   'open_date': None,
   'owner_type_code': None,
   'plus4': None,
   'state': 'CO',
  

In [20]:
url = "https://api.open.fec.gov/v1/schedules/schedule_a/by_employer/?employer=Goldman%20Sachs&per_page=100&api_key=DEMO_KEY&page=12"

In [18]:
response.json()

{'api_version': '1.0',
 'pagination': {'count': 1054, 'page': 12, 'pages': 11, 'per_page': 100},
 'results': []}

In [3]:
def get_url(url):
    data = requests.get(url).json()
    return(data)

In [26]:
test = get_url(url)
print(test)

{'api_version': '1.0', 'pagination': {'page': 12, 'pages': 11, 'per_page': 100, 'count': 1054}, 'results': []}


In [4]:
def get_pages(data):
    pages = data['pagination']['pages']
    return pages

def get_count(data):
    count = data['pagination']['count']
    return count

In [34]:
get_pages(test)

11

In [31]:
get_count(test)

1054

In [ ]:
url = "https://api.open.fec.gov/v1/schedules/schedule_a/by_employer/?employer=Goldman%20Sachs&per_page=100&api_key={}&page={}".format(api_key, page)

In [44]:
pages = get_pages(test)
for page in range(1, pages+1):
    stem = "https://api.open.fec.gov/v1/schedules/schedule_a/by_employer/"
    end = "?employer=Goldman%20Sachs&per_page=100&api_key={}&page={}".format(api_key, page)
    url = stem+end
    data = get_url(url)
    results = data['results']
    print(len(results))

100
100
100
100
100
100
100
100
100
100
54


In [112]:
data = get_url(url)
columns = data['results'][1]
print(type(columns))
for k in columns:
    print(k)
    

columns.keys()


<class 'dict'>
idx
total
cycle
committee_id
count
employer


dict_keys(['idx', 'total', 'cycle', 'committee_id', 'count', 'employer'])

In [119]:
def flattenjson( b, delim ):
    val = {}
    for i in b.keys():
        if isinstance( b[i], dict ):
            get = flattenjson( b[i], delim )
            for j in get.keys():
                val[ i + delim + j ] = get[j]
        else:
            val[i] = b[i]

    return val






In [120]:
# open a file for writing
import csv
results = data['results']
employ_data = open('/tmp/EmployData.csv', 'w')

# create the csv writer object
csv_file = open('test_fec', 'w')
csvwriter = csv.writer(csv_file)

count = 0
for row in results:
    if count == 0:
        header = row.keys()
        csvwriter.writerow(header)
        count += 1
    
    csvwriter.writerow(row.values())

csv_file.close()

In [19]:
def write_csv_json(json_results, filename="jsondata.csv"):

    
    # create the csv and writer
    csv_file = open(filename, 'a')
    csvwriter = csv.writer(csv_file)

    count = 0
    for row in json_results:
        if count == 0:
            header = row.keys()
            csvwriter.writerow(header)
            count += 1

        csvwriter.writerow(row.values())

    csv_file.close()

In [122]:
write_csv_json(results, "mytest.csv")

In [128]:
data = get_url("https://api.open.fec.gov/v1/schedules/schedule_a/?per_page=100&sort=contribution_receipt_date&contributor_employer=Goldman%20Sachs&api_key=DEMO_KEY&two_year_transaction_period=1996")
results = data['results']

In [130]:
write_csv_json(results, "mytest_1996.csv")

In [134]:
data = get_url("https://api.open.fec.gov/v1/schedules/schedule_a/?per_page=100&sort=contribution_receipt_date&contributor_employer=Goldman%20Sachs&api_key=DEMO_KEY&two_year_transaction_period=2008")
results = data['results']
write_csv_json(results, "mytest_2008.csv")

In [41]:
#function to label party from contribtion dest
def get_party_id(committee_id):
    stem = "https://api.open.fec.gov/v1/committee/"
    end = "{}/candidates/?sort=name&page=1&api_key={}&per_page=100".format(committee_id, api_key)
    url = stem+end

    data = get_url(url)
    party_id = data['results'][0]['party']
    office = data['results'][0]['office_full']
    name = data['results'][0]['name']
    print("[*] {}, {}, candidate for {}".format(name, party_id, office))
    return party_id
    
get_party_id("C00464297")
#get_party_id("C00401224") #this one gets error, seems to be no party
#get_party_id("C00571380")

[*] AYOTTE, KELLY A, REP, candidate for Senate


'REP'

In [30]:
#function to label party from contribtion dest
def get_party_ids(committee_id):
    stem = "https://api.open.fec.gov/v1/committee/"
    end = "{}/candidates/?sort=name&page=1&api_key={}&per_page=100".format(committee_id, api_key)
    url = stem+end

    data = get_url(url)
    results = data['results'][0]
    results['committee_id'] = str(committee_id)
    return results

In [ ]:
def get_committee_details(comittee_id)

In [32]:
results = get_party_ids("C00431916")


In [33]:
results

{'active_through': 2008,
 'address_city': 'WILMINGTON',
 'address_state': 'DE',
 'address_street_1': 'PO BOX 438',
 'address_street_2': None,
 'address_zip': '19899',
 'candidate_id': 'P80000722',
 'candidate_inactive': False,
 'candidate_status': 'P',
 'committee_id': 'C00431916',
 'cycles': [1988, 1990, 2008, 2010, 2012],
 'district': '00',
 'district_number': 0,
 'election_districts': ['00', '00'],
 'election_years': [1988, 2008],
 'federal_funds_flag': True,
 'first_file_date': '1987-04-03',
 'flags': 'P80000722',
 'has_raised_funds': True,
 'incumbent_challenge': 'O',
 'incumbent_challenge_full': 'Open seat',
 'last_f2_date': '2007-01-31',
 'last_file_date': '2007-01-31',
 'load_date': '2011-06-21T09:19:07',
 'name': 'BIDEN, JOSEPH R JR',
 'office': 'P',
 'office_full': 'President',
 'party': 'DEM',
 'party_full': 'DEMOCRATIC PARTY',
 'state': 'US'}

In [29]:
type(results)

dict

In [40]:
#https://api.open.fec.gov/v1/committee/C00401224/schedules/schedule_b/by_recipient_id/?per_page=20&api_key=DEMO_KEY&page=1
#returns 1st recipient C00000935
get_party_id("C00000935")

IndexError: list index out of range

In [23]:
get_party_id("C00431916")


[*] BIDEN, JOSEPH R JR, DEM, candidate for President


'DEM'

In [157]:
get_party_id("C00257402")

[*] BONILLA, HENRY, REPUBLICAN PARTY, candidate for House


'REPUBLICAN PARTY'

In [5]:
pages = get_pages(test)
for page in range(1, pages+1):
    stem = "https://api.open.fec.gov/v1/schedules/schedule_a/by_employer/"
    end = "?employer=Goldman%20Sachs&per_page=100&api_key={}&page={}".format(api_key, page)
    url = stem+end
    data = get_url(url)
    results = data['results']
    print(len(results))

NameError: name 'test' is not defined

In [20]:
def req_url_schedule_a(employer, api_key, year=2016, page=1):
    firm = employer.replace(" ", "%20")
    stem = "https://api.open.fec.gov/v1/schedules/schedule_a/?per_page=100&sort=contribution_receipt_date&"
    end = "contributor_employer={}&api_key={}&two_year_transaction_period={}&page={}".format(firm, api_key, year, page)
    url = stem+end
    return url

req_url_schedule_a("Goldman Sachs", api_key)


#results = data['results']
#write_csv_json(results, "mytest_2008.csv")

def get_schedule_a_employer_year(employer, api_key, year=2016):
    firm = employer.replace(" ", "%20")
    
    
    url = req_url_schedule_a(firm, api_key, year)
    data = get_url(url)
    
    #print(data)
    #print(get_pages(data))
    
    pages = get_pages(data)
    for page in range(1, pages+1):
        print("[*] getting FEC SCHEDULE A results for {} in {}...page {} of {}".format(employer, year, page, (pages)))
        filename = "{}_{}_schedule_a.csv".format(year, employer.replace(" ", "_"))
        url = req_url_schedule_a(firm, api_key, year, page)
        page_data = get_url(url)
        results = page_data['results']
        write_csv_json(results, filename)
        
        
        



In [21]:
get_schedule_a_employer_year("Goldman Sachs", api_key)

[*] getting FEC SCHEDULE A results for Goldman Sachs in 2016...page 1 of 37
[*] getting FEC SCHEDULE A results for Goldman Sachs in 2016...page 2 of 37
[*] getting FEC SCHEDULE A results for Goldman Sachs in 2016...page 3 of 37
[*] getting FEC SCHEDULE A results for Goldman Sachs in 2016...page 4 of 37
[*] getting FEC SCHEDULE A results for Goldman Sachs in 2016...page 5 of 37
[*] getting FEC SCHEDULE A results for Goldman Sachs in 2016...page 6 of 37
[*] getting FEC SCHEDULE A results for Goldman Sachs in 2016...page 7 of 37
[*] getting FEC SCHEDULE A results for Goldman Sachs in 2016...page 8 of 37
[*] getting FEC SCHEDULE A results for Goldman Sachs in 2016...page 9 of 37
[*] getting FEC SCHEDULE A results for Goldman Sachs in 2016...page 10 of 37
[*] getting FEC SCHEDULE A results for Goldman Sachs in 2016...page 11 of 37
[*] getting FEC SCHEDULE A results for Goldman Sachs in 2016...page 12 of 37
[*] getting FEC SCHEDULE A results for Goldman Sachs in 2016...page 13 of 37
[*] gett

In [12]:
get_schedule_a_employer_year("Goldman Sachs", api_key, 1996)

10
